In [1]:
import numpy as np
from tensorflow import keras

max_features = 10000  # Only consider the top 20k words
maxlen = 500  # Only consider the first 500 words of each movie review

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [3]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [4]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [5]:
model = models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=maxlen))
model.add(layers.Bidirectional(layers.LSTM(64), merge_mode='concat'))
model.add(layers.Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 1,379,074
Trainable params: 1,379,074
Non-trainable params: 0
_________________________________________________________________


In [6]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
checkpoint_filepath = './temp/checkpoint_bi_lstm_en'
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_weights_only=True, save_best_only=True)

In [8]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.0005), loss='binary_crossentropy', metrics='accuracy')

In [9]:
history=model.fit(x_train, y_train_one_hot, epochs=20, batch_size=128, validation_split=0.1, callbacks=[es, mc])

Epoch 1/20
176/176 [==============================] - 2147s 12s/step - loss: 0.5376 - accuracy: 0.7409 - val_loss: 0.3595 - val_accuracy: 0.8584
Epoch 2/20
176/176 [==============================] - 2516s 14s/step - loss: 0.2978 - accuracy: 0.8832 - val_loss: 0.2888 - val_accuracy: 0.8808
Epoch 3/20
176/176 [==============================] - 1256s 7s/step - loss: 0.2416 - accuracy: 0.9113 - val_loss: 0.3048 - val_accuracy: 0.8828
Epoch 4/20
176/176 [==============================] - 675s 4s/step - loss: 0.2041 - accuracy: 0.9257 - val_loss: 0.3576 - val_accuracy: 0.8460
Epoch 5/20
176/176 [==============================] - 690s 4s/step - loss: 0.1778 - accuracy: 0.9355 - val_loss: 0.3103 - val_accuracy: 0.8852
Epoch 6/20
176/176 [==============================] - 694s 4s/step - loss: 0.1579 - accuracy: 0.9432 - val_loss: 0.3999 - val_accuracy: 0.8696
Epoch 7/20
176/176 [==============================] - 700s 4s/step - loss: 0.1431 - accuracy: 0.9496 - val_loss: 0.3449 - val_accuracy: 0

In [10]:
model.evaluate(x_test, y_test_one_hot)

782/782 [==============================] - 62s 79ms/step - loss: 0.3537 - accuracy: 0.8725


[0.353736013174057, 0.8724799752235413]

아래와 같이 stacked bidirectional LSTM을 구현할 수 있다. <br>
이때, return_sequences=True로 설정해야 한다. 

In [11]:
model1 = models.Sequential()
model1.add(layers.Embedding(max_features, 128, input_length=maxlen))
model1.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
model1.add(layers.Bidirectional(layers.LSTM(64)))
model1.add(layers.Dense(2, activation = 'softmax'))
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 500, 128)         98816     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1,477,890
Trainable params: 1,477,890
Non-trainable params: 0
_________________________________________________________________


In [12]:
model1.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model1.fit(x_train, y_train_one_hot, batch_size=32, epochs=2, validation_split=0.2)

Epoch 1/2
625/625 [==============================] - 521s 827ms/step - loss: 0.5448 - accuracy: 0.7265 - val_loss: 0.4680 - val_accuracy: 0.8026
Epoch 2/2
625/625 [==============================] - 502s 804ms/step - loss: 0.3377 - accuracy: 0.8598 - val_loss: 0.3284 - val_accuracy: 0.8554


In [13]:
model1.evaluate(x_test, y_test_one_hot)

782/782 [==============================] - 142s 181ms/step - loss: 0.3383 - accuracy: 0.8526


[0.3383167088031769, 0.8525999784469604]